In [17]:
#Fetch and set up version 0.28 of the OpenAI Python library
!pip install openai==0.28

In [3]:
import time
import pandas as pd
import openai
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [5]:
#Ask for the location 

user_input = input("Enter the location to search on Google Maps (be as specific as possible, e.g. Ateneo de Manila University, Katipunan Avenue, Quezon City): ")

Enter the location to search on Google Maps (be as specific as possible, e.g. Ateneo de Manila University, Katipunan Avenue, Quezon City):  Taft


In [7]:

# Initialize the browser
browser = webdriver.Chrome()

# Navigate to Google Maps
browser.get('https://www.google.com/maps')

# Wait for the page to load
time.sleep(3)

# Find the search bar and enter the user input
search_bar = browser.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_bar.send_keys(user_input)

# Wait for the suggestions to load and appear
time.sleep(2)

# Find and click the first suggestion
try:
    first_suggestion = browser.find_element(By.CLASS_NAME, 'DgCNMb')
    first_suggestion.click()
except:
    time.sleep(1)
# Wait for the place to load and appear
time.sleep(2)

# store the address for future use (if there is)

loc_address = browser.find_element(By.CSS_SELECTOR, 'span.DkEaL').text

# Click Restaurant Option
restaurants_option = browser.find_element(By.XPATH, '//button[@aria-label="Restaurants"]')
restaurants_option.click()

# Wait for all restaurants to load and appear
time.sleep(2)

#filter 3.5*only
rating = browser.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[1]/button/div/span[2]')
rating.click()

time.sleep(2)

rating = browser.find_element(By.XPATH, '//*[@id="action-menu"]/div[5]')
rating.click()
time.sleep(2)

#Zoom out function so that the search area button works
zoom_out = browser.find_element(By.XPATH, '//*[@id="widget-zoom-out"]/div')
zoom_out.click()

time.sleep(1)

#Search Area button
search_area = browser.find_element(By.XPATH, '//*[@id="search-this-area"]/div/button/span')
search_area.click()

time.sleep(2)

# Initialize a dictionary to store restaurant details
restaurants_dict = {}

# Scroll and gather restaurant details
for i in range(4):
    # Get all restaurant elements
    restaurant_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(restaurant_list[len(restaurant_list)-1])
    action.scroll_from_origin(scroll_origin, 0, 2000).perform()
    time.sleep(1)
    # new_restaurant_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    # if len(new_restaurant_list) == len(restaurant_list):
    #     break

#clicks each element then extracts corresponding info
for i in range(len(restaurant_list)):
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(restaurant_list[i])
    action.scroll_from_origin(scroll_origin, 0, 150).perform()
    time.sleep(1)
    action.move_to_element(restaurant_list[i]).perform()
    restaurant_list[i].click()
    time.sleep(3)
    
    try:
        review_element = WebDriverWait(browser, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[aria-label*="review"]'))
        )
            # Extract the number of reviews from the text
    except:
        continue
        
        # Extract the number of reviews from the text
    num_of_reviews = int(review_element.text[1:-1].replace(',', ''))
    if num_of_reviews <= 65:
        continue
        
    #Restaurant Name
    restaurant_name = browser.find_element(By.CSS_SELECTOR, '.a5H0ec').find_element(By.XPATH, './..').text
    
    #Restaurant Rating
    restaurant_rating_element = WebDriverWait(browser, 2).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]'))
    )
    restaurant_rating = restaurant_rating_element.text
    #Restaurant Address
    restaurant_address = browser.find_element(By.CLASS_NAME, 'Io6YTe.fontBodyMedium.kR99db').text

    #Restaurant Contact Number
    try: 
        restaurant_contact = browser.find_element(By.CSS_SELECTOR, 'button[data-item-id^="phone:"] .Io6YTe.fontBodyMedium.kR99db').text
    except: 
        restaurant_contact = "N/A"

    #Restaurant Hours
    try:
        restaurant_hours = browser.find_element(By.CSS_SELECTOR, 'span.ZDu9vd').text
        if "Closes" in restaurant_hours:
            restaurant_hours = restaurant_hours.replace('\u202f', ' ').strip()[7:]
            
    except:
        try:
            restaurant_hours = browser.find_element(By.CSS_SELECTOR, 'div[aria-label="Open 24 hours"]').text
        except:
            restaurant_hours = "N/A"
    restaurants_dict[restaurant_name] = {'Rating': restaurant_rating, 'Address': restaurant_address, 'Contact': restaurant_contact, 'Operating Hours': restaurant_hours}

#Go back to the previous page (location view)
back = browser.find_element(By.XPATH, '//*[@id="searchbox"]/div[2]/div/button/span')
back.click()

# Wait for the page to navigate back
time.sleep(2)

# Explicitly wait for the "Things to do" option to be clickable
things_to_do_option = browser.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[3]/button/div/span[1]')
things_to_do_option.click()

# Wait for all things to do to appear
time.sleep(2)

#filter 3.5*only
rating = browser.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[1]/button/div/span[2]')
rating.click()

time.sleep(2)

rating = browser.find_element(By.XPATH, '//*[@id="action-menu"]/div[5]')
rating.click()
time.sleep(3)

#Zoom out function so that the search area button works
zoom_out = browser.find_element(By.XPATH, '//*[@id="widget-zoom-out"]/div')
zoom_out.click()

time.sleep(2)

#Search Area button
search_area = browser.find_element(By.XPATH, '//*[@id="search-this-area"]/div/button/span')
search_area.click()

time.sleep(2)

# Initialize a dictionary to store things to do details
things_to_do_dict = {}

# Scroll and gather things to do details
for i in range(4):
    # Get all things to do elements
    things_to_do_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(things_to_do_list[len(things_to_do_list)-1])
    action.scroll_from_origin(scroll_origin, 0, 2000).perform()
    time.sleep(3)
    # new_things_to_do_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    # if len(new_things_to_do_list) == len(things_to_do_list):
    #     break

action = ActionChains(browser)
#clicks each element then extracts info
for i in range(len(things_to_do_list)):
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(things_to_do_list[i])
    action.scroll_from_origin(scroll_origin, 0, 200).perform()
    time.sleep(1)
    action.move_to_element(things_to_do_list[i]).perform()
    things_to_do_list[i].click()
    time.sleep(3)

    #Activity Reviews
    try:
        review_element = WebDriverWait(browser, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[aria-label*="review"]'))
            )
            # Extract the number of reviews from the text
    except:
        continue
        
        # Extract the number of reviews from the text
    num_of_reviews = int(review_element.text[1:-1].replace(',', ''))
    if num_of_reviews <= 100:
        continue
        
    #Activity Name
    activity_name = browser.find_element(By.CSS_SELECTOR, '.a5H0ec').find_element(By.XPATH, './..').text

    #Activity Rating
    activity_rating_element = WebDriverWait(browser, 2).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]'))
        )
    activity_rating = activity_rating_element.text
    #Activity Address
    activity_address = browser.find_element(By.CLASS_NAME, 'Io6YTe.fontBodyMedium.kR99db').text

    #Activity Contact
    try: 
        activity_contact = browser.find_element(By.CSS_SELECTOR, 'button[data-item-id^="phone:"] .Io6YTe.fontBodyMedium.kR99db').text
    except: 
        activity_contact = "N/A"

    #Activity Hours
    try:
        activity_hours = browser.find_element(By.CSS_SELECTOR, 'span.ZDu9vd').text
        if "Closes" in activity_hours:
            activity_hours = activity_hours.replace('\u202f', ' ').strip()[7:]
            
    except:
        try:
            activity_hours = browser.find_element(By.CSS_SELECTOR, 'div[aria-label="Open 24 hours"]').text
        except:
            activity_hours = "N/A"

    things_to_do_dict[activity_name] = {'Rating': activity_rating, 'Address': activity_address, 'Contact': activity_contact, 'Operating Hours': activity_hours}

browser.back()
browser.back()

time.sleep(2)
search_bar.clear()

# Type in the search bar to search for shopping malls
if loc_address.count(',') > 1:
    search_bar.send_keys("Shopping Malls near " + loc_address)

else:
    search_bar.send_keys("Shopping Malls near " + user_input)

time.sleep(1)

# Press the search button
search = browser.find_element(By.XPATH, '//*[@id="searchbox-searchbutton"]/span')
search.click()

time.sleep(3)

#filter 3.5*only
rating = browser.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[1]/button/div/span[2]')
rating.click()

time.sleep(2)

rating = browser.find_element(By.XPATH, '//*[@id="action-menu"]/div[5]')
rating.click()
time.sleep(2)

# Wait for the results to load
time.sleep(2)

# Initialize a dictionary to store shopping mall details
shopping_mall_dict = {}

# Scroll and gather things to do details
for i in range(4):
    # Get all mall elements
    shopping_mall_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(shopping_mall_list[len(shopping_mall_list)-1])
    action.scroll_from_origin(scroll_origin, 0, 2000).perform()
    time.sleep(1)
    # new_shopping_mall_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    # if len(new_shopping_mall_list) == len(shopping_mall_list):
    #     break

action = ActionChains(browser)
#clicks each element then extracts info
for i in range(len(shopping_mall_list)):
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(shopping_mall_list[i])
    action.move_to_element(shopping_mall_list[i]).perform()
    shopping_mall_list[i].click()
    time.sleep(2)

    #Shopping mall Reviews
    try:
        review_element = WebDriverWait(browser, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[aria-label*="review"]'))
            )
            # Extract the number of reviews from the text
    except:
        continue
        
        # Extract the number of reviews from the text
    num_of_reviews = int(review_element.text[1:-1].replace(',', ''))
    if num_of_reviews <= 1000:
        continue
    #Shopping Mall Name
    shopping_mall_name = browser.find_element(By.CSS_SELECTOR, '.a5H0ec').find_element(By.XPATH, './..').text

    #Shopping Mall Rating
    shopping_mall_rating_element = WebDriverWait(browser, 5).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]'))
    )
    shopping_mall_rating = shopping_mall_rating_element.text
    #Shopping Mall Address
    shopping_mall_address = browser.find_element(By.CLASS_NAME, 'Io6YTe.fontBodyMedium.kR99db').text

    #Shopping Mall Contact
    try: 
        shopping_mall_contact = browser.find_element(By.CSS_SELECTOR, 'button[data-item-id^="phone:"] .Io6YTe.fontBodyMedium.kR99db').text
    except: 
        shopping_mall_contact = "N/A"

    #Shopping Mall Hours
    try:
        shopping_mall_hours = browser.find_element(By.CSS_SELECTOR, 'span.ZDu9vd').text
        if "Closes" in shopping_mall_hours:
            shopping_mall_hours = activity_hours.replace('\u202f', ' ').strip()[7:]
            
    except:
        try:
            shopping_mall_hours = browser.find_element(By.CSS_SELECTOR, 'div[aria-label="Open 24 hours"]').text
        except:
            shopping_mall_hours = "N/A"

    shopping_mall_dict[shopping_mall_name] = {'Rating': shopping_mall_rating, 'Address': shopping_mall_address, 'Contact': shopping_mall_contact, 'Operating Hours': shopping_mall_hours}

# Close the browser
browser.quit()


In [29]:
# Convert the restaurants dictionary to a DataFrame and add a new column for type
df_restaurants = pd.DataFrame.from_dict(restaurants_dict, orient='index')
df_restaurants.reset_index(inplace=True)
df_restaurants.rename(columns={'index': 'Name'}, inplace=True)
df_restaurants['Type'] = 'Restaurant'
display(df_restaurants)
# Asks if there are specific locations the person wants to visit and how many restaurants to visit
# If not given enough specific locations, it will perform a random search
no_restaurant = int(input("How many food spots do you want to visit?"))
while no_restaurant > len(df_restaurants):
    print("Not enough locations to support the number")
    no_restaurant = int(input("How many food spots do you want to visit?"))

list_restaurants = []
specific_restaurant = ''

# Loop to gather specific restaurants requested by the user
while  len(list_restaurants) < no_restaurant:
    specific_restaurant = input('Any specific restaurants to visit? (quit if done)')
    
    if specific_restaurant.upper() == 'QUIT':
        break
        
    if not df_restaurants[df_restaurants['Name'] ==  specific_restaurant].empty:
        
        # Extract details of the specific restaurant and add to list
        row = df_restaurants[df_restaurants["Name"] == specific_restaurant].to_dict()
        key = list(row['Name'].keys())[0]
        restaurant = "Name: " + row['Name'][key] + "\nRating: " + row['Rating'][key] + "\nAddress: " + row['Address'][key] + "\nContact: " + row['Contact'][key] + "\nHours: " + row['Operating Hours'][key]
        list_restaurants.append(restaurant)
        
        # Remove selected restaurant from DataFrame
        df_restaurants = df_restaurants.drop(index = key)
        
    else:
        print('Location is not in the dataframe')
        
#Subtract the number of specific locations to the number of locations wanted
no_restaurant = no_restaurant - len(list_restaurants)

# Randomly sample remaining restaurants from DataFrame
rows = df_restaurants.sample(n=no_restaurant)

# Loop to gather randomly selected restaurants 
for key in rows.get('Name').keys():
    restaurant = "Name: " + rows['Name'][key] + "\nRating: " + rows['Rating'][key] + "\nAddress: " + rows['Address'][key] + "\nContact: " + rows['Contact'][key] + "\nHours: " + rows['Operating Hours'][key]

    # Combine specific and randomly selected restaurants into one list
    list_restaurants.append(restaurant)
    
list_restaurants

,Name,Rating,Address,Contact,Operating Hours,Type
0,McDonald's Quirino,4.0,"Ruby Ann Building, San Andres St, Malate, Mani...",(02) 8888 6236,Open 24 hours,Restaurant
1,Kanto Freestyle Breakfast La Salle,4.4,"2466 Leon Guinto St, Malate, Manila, 1004 Metr...",0919 478 8055,Open 24 hours,Restaurant
2,Thai Mango - Malate,3.6,"2463 Leon Guinto Manila Malate, 1004 Metro Manila",0945 870 6519,Open 24 hours,Restaurant
3,Army Navy Burger + Burrito,4.1,"Burgundy Transpacific Place, Unit G-6, No. 245...",0968 548 4514,Open 24 hours,Restaurant
4,Tapa King Taft Avenue,3.9,"1st Flr., Commercial Unit No. 5, One Archers P...",(02) 8888 8272,Open 24 hours,Restaurant
5,El Poco Cantina,4.7,"945 Estrada St, Malate, Manila, 1004 Metro Manila",0920 980 0945,Closed ⋅ Opens 10 AM,Restaurant
6,Angrydobo,3.7,"2456, 1004 Taft Ave, Malate, Manila, 1004 Metr...",0968 619 4971,Closed ⋅ Opens 10 AM,Restaurant
7,Chomp Chomp,4.3,"2nd Floor, Bellago Residence, 2450 Leon Guinto...",(02) 8775 7028,Closed ⋅ Opens 10 AM,Restaurant
8,Zark's Burgers,4.2,"2464 Taft Ave, Malate, Manila, 1000 Metro Manila",(02) 3392 1211,Closed ⋅ Opens 10 AM,Restaurant
9,Tinuhog Ni Benny Barbecue House,4.4,"879 Dagonoy St, Malate, Manila, 1004 Metro Manila",(02) 8253 1072,Closed ⋅ Opens 9:30 AM,Restaurant


How many food spots do you want to visit? 2
Any specific restaurants to visit? (quit if done) Ababu
Any specific restaurants to visit? (quit if done) quit


['Name: Ababu\nRating: 3.9\nAddress: 931 Estrada St, Malate, Manila, 1004 Metro Manila\nContact: N/A\nHours: Closed ⋅ Opens 10\u202fAM',
 'Name: Kenny Rogers Roasters\nRating: 4.2\nAddress: 2576 Taft Ave, Malate, Manila, 1004 Metro Manila\nContact: (02) 8555 9000\nHours: Closed ⋅ Opens 8\u202fAM']

In [9]:
# Convert the things to do dictionary to a DataFrame and add a new column for type
df_things_to_do = pd.DataFrame.from_dict(things_to_do_dict, orient='index')
df_things_to_do.reset_index(inplace=True)
df_things_to_do.rename(columns={'index': 'Name'}, inplace=True)
df_things_to_do['Type'] = 'Attraction'
display(df_things_to_do)
# Asks if there are specific locations the person wants to visit and how many attractions to visit
# If not given enough specific locations, it will perform a random search
no_attraction = int(input("How many attractions do you want to visit?"))
while no_attraction > len(df_things_to_do):
    print("Not enough locations to support the number")
    no_attraction = int(input("How many attractions do you want to visit?"))
    
list_attractions = []
specific_attraction = ''

# Loop to gather specific attractions requested by the user
while  len(list_attractions) < no_attraction:
    specific_attraction = input('Any specific attractions to visit? (quit if done)')
    
    if specific_attraction.upper() == 'QUIT':
        break
        
    if not df_things_to_do[df_things_to_do['Name'] ==  specific_attraction].empty:
        
        # Extract details of the specific things to do and add to list
        row = df_things_to_do[df_things_to_do["Name"] == specific_attraction].to_dict()
        key = list(row['Name'].keys())[0]
        attraction = "Name: " + row['Name'][key] + "\nRating: " + row['Rating'][key] + "\nAddress: " + row['Address'][key] + "\nContact: " + row['Contact'][key] + "\nHours: " + row['\nOperating Hours'][key]
        list_attractions.append(attraction)
        
        # Remove selected things to do from DataFrame
        df_things_to_do = df_things_to_do.drop(index = key)
        
    else:
        print('Location is not in the dataframe')
        
#Subtract the number of specific locations to the number of locations wanted
no_attraction = no_attraction - len(list_attractions)

# Randomly sample remaining things to do from DataFrame
rows = df_things_to_do.sample(n=no_attraction)

# Loop to gather randomly selected things to do 
for key in rows.get('Name').keys():
    attraction = "Name: " + rows['Name'][key] + "\nRating: " + rows['Rating'][key] + "\nAddress: " + rows['Address'][key] + "\nContact: " + rows['Contact'][key] + "\nHours: " + rows['Operating Hours'][key]
    
    # Combine specific and randomly selected things to do into one list
    list_attractions.append(attraction)
    
list_attractions

,Name,Rating,Address,Contact,Operating Hours,Type
0,Fort Santiago,4.6,"HXVC+M5V, Intramuros, Manila, 1002 Metro Manila",(02) 8527 3155,Closed ⋅ Opens 8 AM,Attraction
1,Manila Ocean Park,4.2,"Quirino Grandstand, 666 Behind, Ermita, Manila...",(02) 7238 6188,Closed ⋅ Opens 10 AM,Attraction
2,Rizal Park,4.5,"Ermita, Manila, 1000 Metro Manila",N/A,Opens soon ⋅ 5 AM,Attraction
3,National Museum of Fine Arts,4.7,"Padre Burgos Ave, Ermita, Manila, 1000 Metro M...",(02) 8527 1215,Closed ⋅ Opens 9 AM,Attraction
4,Ayala Museum,4.6,"Greenbelt Park, Makati Avenue, corner Dela Ros...",0917 834 3845,Closed ⋅ Opens 10 AM,Attraction
5,National Museum of Natural History,4.7,"Teodoro F. Valencia Cir, Ermita, Manila, 1000 ...",(02) 8298 1100,Closed ⋅ Opens 9 AM,Attraction
6,Ayala Triangle Gardens,4.5,"Paseo De Roxas St Cor Makati Ave, Cor Ayala Av...",(02) 7908 3000,Closed ⋅ Opens 6 AM,Attraction
7,Manila Baywalk Dolomite Beach,4.2,"Roxas Blvd, Malate, Manila, 1004 Metro Manila",N/A,Closed ⋅ Opens 6 AM,Attraction
8,Arroceros Forest Park,4.5,"659 A Antonio Villegas St, Ermita, Manila, 100...",0917 848 1004,Closed ⋅ Opens 8 AM,Attraction
9,Washington Sycip Park,4.6,"Legazpi Street, Legaspi Village, Makati, 1229 ...",N/A,Closed ⋅ Opens 6 AM,Attraction


KeyboardInterrupt: Interrupted by user

In [13]:
# Convert the shopping malls dictionary to a DataFrame and add a new column for type
df_shopping_mall = pd.DataFrame.from_dict(shopping_mall_dict, orient='index')
df_shopping_mall.reset_index(inplace=True)
df_shopping_mall.rename(columns={'index': 'Name'}, inplace=True)
df_shopping_mall['Type'] = 'Shopping'
display(df_shopping_mall)

# Asks if there are specific shopping mall the person wants to visit and how many shopping malls to visit
# If not given enough specific locations, it will perform a random search
no_mall = int(input("How many shopping malls do you want to visit?"))
while no_mall > len(df_shopping_mall):
    print("Not enough locations to support the number")
    no_mall = int(input("How many shopping malls do you want to visit?"))
    
list_shopping_mall = []
specific_mall = ''

# Loop to gather specific shopping malls requested by the user
while  len(list_shopping_mall) < no_mall:
    specific_mall = input('Any specific malls to visit? (quit if done)')
    
    if specific_mall.upper() == 'QUIT':
        break
        
    if not df_shopping_mall[df_shopping_mall['Name'] ==  specific_mall].empty:
        
         # Extract details of the specific things to do and add to list
        row = df_shopping_mall[df_shopping_mall["Name"] == specific_mall].to_dict()
        key = list(row['Name'].keys())[0]
        mall = "Name: " + row['Name'][key] + "\nRating: " + row['Rating'][key] + "\nAddress: " + row['Address'][key] + "\nContact: " + row['Contact'][key] + "\nHours: " + row['\nOperating Hours'][key]
        list_shopping_mall.append(mall)

        # Remove selected things to do from DataFrame
        df_shopping_mall = df_shopping_mall.drop(index = key)
        
    else:
        print('Location is not in the dataframe')
        
#Subtract the number of specific locations to the number of locations wanted
no_mall = no_mall - len(list_shopping_mall)

# Randomly sample remaining things to do from DataFrame
rows = df_shopping_mall.sample(n=no_mall)

# Loop to gather randomly selected things to do 
for key in rows.get('Name').keys():
    mall = "Name: " + rows['Name'][key] + "\nRating: " + rows['Rating'][key] + "\nAddress: " + rows['Address'][key] + "\nContact: " + rows['Contact'][key] + "\nHours: " + rows['Operating Hours'][key]
     
    # Combine specific and randomly selected things to do into one list
    list_shopping_mall.append(mall)
    
list_shopping_mall
#this list_full is what we input into the chat gpt api

,Name,Rating,Address,Contact,Operating Hours,Type
0,Two Shopping Center,4.2,"Brgy 79 Zone 10 Taft Ave Ext, corner Cuneta Av...",0915 650 7654,Closed ⋅ Opens 8 AM,Shopping
1,Robinsons Place Manila,4.6,"Pedro Gil, cor Adriatico St, Ermita, Manila, 1...",(02) 8310 3333,Closed ⋅ Opens 10 AM,Shopping
2,VICTORY PASAY MALL,3.9,"Antonio S. Arnaiz Ave, Pasay, Metro Manila",0917 210 8353,Closed ⋅ Opens 10 AM,Shopping
3,Cartimar Shopping Center,4.2,"Cartimar Ave, Tai Kei Leng, Pasay, Metro Manila",0916 988 7384,Closed ⋅ Opens 7 AM,Shopping


How many shopping malls do you want to visit? 1
Any specific malls to visit? (quit if done) quit


['Name: VICTORY PASAY MALL\nRating: 3.9\nAddress: Antonio S. Arnaiz Ave, Pasay, Metro Manila\nContact: 0917 210 8353\nHours: Closed ⋅ Opens 10\u202fAM']

In [37]:
# Setting the OpenAI API key for authentication
openai.api_key = ''

def generate_itinerary(restaurants, attractions, shopping_malls, time1, time2):
    # Prompt for generating the itinerary
    prompt = ("Create a one-day itinerary lasting from " +time1+ " to " +time2+ " using all of the locations within the following lists of places:\n\n"
        "Restaurants:\n\n" + "\n\n".join(restaurants) + "\n\n"
        "Attractions:\n\n" + "\n\n".join(attractions) + "\n\n"
        "Shopping Malls:\n\n" + "\n\n".join(shopping_malls) + "\n\n")

    # Generate itinerary and get response using OpenAI's ChatGPT model
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0.7
    )
    # Return the generated itinerary
    return print(response['choices'][0]['message']['content'].strip())

# Collect input for trip start and end times
time1 = input("What time does the trip start? (00:00 24 hour format)")
time2 = input("What time does the trip end? 00:00 24 hour format)")


generate_itinerary(list_restaurants, list_attractions, list_shopping_mall, time1, time2)

What time does the trip start? (00:00 24 hour format) 10:00
What time does the trip end? 00:00 24 hour format) 18:00


10:00 AM - Start your day by visiting The Dessert Museum in Pasay. Indulge in a sweet experience and take some Instagram-worthy photos.

12:00 PM - Head over to Ababu for a delicious lunch. Try their specialties and enjoy the cozy atmosphere.

2:00 PM - After lunch, take a leisurely stroll along Manila Baywalk Dolomite Beach. Enjoy the scenic views of the bay and relax by the water.

4:00 PM - Make your way to Victory Pasay Mall for some shopping. Explore the shops and find some souvenirs to take home.

6:00 PM - End your day with a hearty dinner at Kenny Rogers Roasters. Enjoy a delicious meal before heading back to your accommodation.
